In [1]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, model_outputs):
        results = super().postprocess(
            model_outputs=model_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        return np.unique([result.get("word").strip() for result in results])

In [2]:
# Load pipeline
model_name = "ml6team/keyphrase-extraction-kbir-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

Downloading:   0%|          | 0.00/855 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [33]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [58]:
wordnet_lemmatizer.lemmatize('Statins')

'Statins'

In [126]:
import re
def clean_data(text):
    clean_text = re.sub("@[A-Za-z0-9_ ]+","", text)
    clean_text = re.sub("#[A-Za-z0-9_ ]+","", clean_text)
    clean_text = clean_text.replace('RT ','')
    nltk_tokens = nltk.word_tokenize(clean_text)
    word_list=[]
    for w in nltk_tokens:
        word_list.append(wordnet_lemmatizer.lemmatize(w.lower()))
    text = ' '.join(word_list)
    return text

In [133]:
def get_phrases(text):
    text = clean_data(text)
    keyphrases = extractor(text.replace("\n", " "))
    return ', '.join(keyphrases)

In [82]:
keyphrases = extractor(text.replace("\n", " "))

print(keyphrases)

['apheresis' 'ascvd']


In [87]:
keyphrases = extractor(clean_data(text.replace("\n", " ")))

print(keyphrases)

['age' 'aging' 'astatin' 'ch' 'crest' 'diabetes' 'family history' 'heart'
 'high blood pressure' 'inflammation' 'nitrous oxide'
 'patient cardivascular risk' 'plotropic effect' 'ro' 'rosu' 'side effect'
 'smoking' 'statin' 'statin medication' 'statin therapy' 'sterol']


In [42]:
!pip install yake

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [43]:
import yake

In [124]:
language = "en"
max_ngram_size = 1
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1

def process_yake(text):
    text = clean_data(text)
    if len(text)>1000:
        numOfKeywords = 20
        custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
        keywords = custom_kw_extractor.extract_keywords((text))
    elif len(text)>200 and len(text)<1000:
        numOfKeywords = 10
        custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
        keywords = custom_kw_extractor.extract_keywords((text))
    else:
        numOfKeywords = 5
        custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
        keywords = custom_kw_extractor.extract_keywords((text))

    key_values=[]    
    for kw in keywords:
        key_values.append(kw[0])
    return ', '.join(key_values)

In [125]:
process_yake(text)

'statin, risk, cholesterol, lower, heart, attack, effect, plaque, vessel, stroke, level, crestor, blood, medication, high, diabetes, number, rosuvastatin, atorvastatin, equivalent'

In [176]:
import pandas as pd
data = pd.read_excel('C:/Users/GOWRICH1/Downloads/Leqvio_Lexicon analysis01.19.23.xlsx',sheet_name='HCP')
data

Region                              Content/Full verbatim    Source  \
0         US                                                NaN  Facebook   
1    Germany  @hut_german @LelLAlLL @m_grill @dgn_ev @BMBF_B...   Twitter   
2    Austria  #praluent #pcsk9hemmer 5 years after approval,...   Twitter   
3         UK  RT @eatmostlyfatali @robertquickert @MaryanneD...   Twitter   
4         UK   @RaberLorenz @SABOURETCardio @PCRonline @esca...   Twitter   
..       ...                                                ...       ...   
851       US  RT @CardiologyToday #Evolocumab, a PCSK9 inhib...   Twitter   
852    Spain  RT @secardiologia .@acorderofort in #RiesgoSEC...   Twitter   
853       US  RT @Drlipid Keep in mind the dynamics of PSCK9...   Twitter   
854       US  RT @lipiddoc Love the  8 page prior auth form ...   Twitter   
855    Spain  Alirocumab reduced MACE in patientswithout a h...   Twitter   

                 Published Date Reporter type  
0                          2018           HCP  
1                          2022           HCP  
2                          2022           HCP  
3    2020-10-19 13:15:13.228000           HCP  
4    2021-04-03 12:21:37.941000           HCP  
..                          ...           ...  
851  2021-01-20 16:50:57.177000           HCP  
852  2021-10-05 22:05:30.709000           HCP  
853  2022-08-16 16:07:38.265000           HCP  
854  2022-01-12 18:29:43.253000           HCP  
855  2021-12-28 13:04:09.520000           HCP  

[856 rows x 5 columns]

In [177]:
data = data.replace(np.nan,'')
data

Region                              Content/Full verbatim    Source  \
0         US                                                     Facebook   
1    Germany  @hut_german @LelLAlLL @m_grill @dgn_ev @BMBF_B...   Twitter   
2    Austria  #praluent #pcsk9hemmer 5 years after approval,...   Twitter   
3         UK  RT @eatmostlyfatali @robertquickert @MaryanneD...   Twitter   
4         UK   @RaberLorenz @SABOURETCardio @PCRonline @esca...   Twitter   
..       ...                                                ...       ...   
851       US  RT @CardiologyToday #Evolocumab, a PCSK9 inhib...   Twitter   
852    Spain  RT @secardiologia .@acorderofort in #RiesgoSEC...   Twitter   
853       US  RT @Drlipid Keep in mind the dynamics of PSCK9...   Twitter   
854       US  RT @lipiddoc Love the  8 page prior auth form ...   Twitter   
855    Spain  Alirocumab reduced MACE in patientswithout a h...   Twitter   

                 Published Date Reporter type  
0                          2018           HCP  
1                          2022           HCP  
2                          2022           HCP  
3    2020-10-19 13:15:13.228000           HCP  
4    2021-04-03 12:21:37.941000           HCP  
..                          ...           ...  
851  2021-01-20 16:50:57.177000           HCP  
852  2021-10-05 22:05:30.709000           HCP  
853  2022-08-16 16:07:38.265000           HCP  
854  2022-01-12 18:29:43.253000           HCP  
855  2021-12-28 13:04:09.520000           HCP  

[856 rows x 5 columns]

In [178]:
data['keywords'] = data['Content/Full verbatim'].apply(process_yake)

In [179]:
data['keywords1'] = data['Content/Full verbatim'].apply(get_phrases)

In [170]:
data['Reporter type'].value_counts()

Patient      1474
Caregiver      79
Name: Reporter type, dtype: int64

In [172]:
patient_data = data[data['Reporter type']=='Patient']
patient_data

Region                              Content/Full verbatim  \
0           Spain  Hello... one question. I'm looking for pregnan...   
2           Spain  Been in treatment since February. The business...   
3           Spain  I give faith to Praluent .... Cholesterol leve...   
4           Spain  I also injected myself with ALIROCUMAB 75mg an...   
5           Spain  Can you tell me what they're called, I've been...   
...           ...                                                ...   
1548  Netherlands  i stopped taking it because of side effects an...   
1549  Netherlands  Used for a year and a half now stopped that wa...   
1550  Netherlands  It has now been 7 years since I got 4 redirect...   
1551  Netherlands  Just started so no experience yet. However, th...   
1552  Netherlands  After 2x myocardial infarction, permanent high...   

            Source Published Date Reporter type  \
0         Facebook           2018       Patient   
2         Facebook           2018       Patient   
3         Facebook           2018       Patient   
4         Facebook           2018       Patient   
5         Facebook           2018       Patient   
...            ...            ...           ...   
1548  Reviews Type     15-01-2019       Patient   
1549  Reviews Type     06-01-2019       Patient   
1550  Reviews Type     04-01-2019       Patient   
1551  Reviews Type     10-12-2018       Patient   
1552  Reviews Type     07-12-2018       Patient   

                                               keywords  \
0     cholesterol, question, statin, pregnancy, removed   
2     treatment, cholesterol, february, repatha, pri...   
3     praluent, give, faith, cholesterol, total, yea...   
4     alirocumab, ezetimiba, atozet, worth, injected...   
5         praulent, called, prescribed, injectable, pen   
...                                                 ...   
1548  effect, side, ldl, early, case, pharma, stoppe...   
1549  year, work, lot, cholesterol, case, weak, atta...   
1550  year, twelve, week, cardiologist, repatha, tol...   
1551        day, started, experience, muscle, complaint   
1552  side, effect, cholesterol, myocardial, infarct...   

                                              keywords1  
0          290 cholesterol, family hypercholesterolemia  
2     contraindication, disabling muscle pain, lipid...  
3                 atin, cholesterol level, heart attack  
4                      atozet, ezetimiba, rosuvastatina  
5                              injectable pen, praulent  
...                                                 ...  
1548                       ldl cholesterol, side effect  
1549                 heart attack, muscle pain, syringe  
1550                card, cholesterol, pepatha, repatha  
1551                          ezetrol, muscle complaint  
1552              2x myocardial infarction, side effect  

[1474 rows x 7 columns]

In [171]:
cargiver_data = data[data['Reporter type']=='Caregiver']
cargiver_data

Region                              Content/Full verbatim  \
1           Spain  My husband use repatha and it change his life\...   
8           Spain  In Valladolid PCSK9 Inhibitors is and at the m...   
18             US  My daughter 13 and myself both have hashimotos...   
126            UK  My wife and children have FH and they have bee...   
147            UK  Please don’t panic. Statins generally work ver...   
...           ...                                                ...   
1352           US  Been on them for about 10 years, if the doc sa...   
1354           US  I don't see the point of bringing your LDL dow...   
1371           UK  So hard, if like my Dads situation he is havin...   
1386           US  RT @mdavidsonmd My FH patient for the past 25 ...   
1402  Netherlands  Search on wikipedia. cholesterol. Serious doub...   

            Source              Published Date Reporter type  \
1         Facebook                        2018     Caregiver   
8         Facebook                        2017     Caregiver   
18        Facebook                        2016     Caregiver   
126         Forums         2021-07-19 09:36:00     Caregiver   
147         Forums         2021-02-12 19:47:00     Caregiver   
...            ...                         ...           ...   
1352        Forums         2022-04-14 11:03:08     Caregiver   
1354        Forums         2022-04-13 03:59:53     Caregiver   
1371        Forums         2020-09-16 10:53:00     Caregiver   
1386       Twitter  2020-12-12 14:14:45.889000     Caregiver   
1402  Reviews Type                  20-06-2018     Caregiver   

                                               keywords  \
1               rapatha, husband, repatha, change, life   
8        valladolid, inhibitor, moment, husband, denied   
18              daughter, cholesterol, hashimotos, high   
126                 week, good, wife, child, prescribed   
147   genetic, test, dad, blood, statin, don, panic,...   
...                                                 ...   
1352  quote, year, heart, wobbly, post, member, simv...   
1354  repatha, sound, good, frighten, doe, agee, sup...   
1371  thing, hard, dad, psp, found, cholesterol, wor...   
1386  age, vifuwjjrhg, year, evolocumab, ldl-c, card...   
1402  cholesterol, disease, father, doubt, lowering,...   

                                              keywords1  
1                                      rapatha, repatha  
8                    cardiologist, end, pcsk9 inhibitor  
18                         hashimotos, high cholesterol  
126                                   repatha injection  
147                          genetic blood test, statin  
...                                                 ...  
1352  atorvastatin, heart attack, heart bypass, qual...  
1354                cholesterol, lipoprotein a, repatha  
1371                                               stat  
1386              65, preventive cardiologist, xanthoma  
1402  cancer, elderly, pharmaceutical industry, repa...  

[79 rows x 7 columns]

In [180]:
phrase_list = list(filter(('').__ne__, data['keywords1'].to_list()))
phrase_list

['diabetes, guideline development, heart attack, public fund, stroke',
 'plaque regression, surrogate endpoint',
 'repatha',
 'antihistamine, inclisiran, pruritis',
 'cardiovascular disease, chemical reaction, dental damage, diabetes, dietitian, healthy eating, heart, internet, margarine, monounsaturated fat, obesity, saturated fat, sedentary behavior, trans fatty acid, type 2 diabetes',
 'cholesterol, statin',
 'lovastatin, red rice',
 'cardiovascular disease conference, inclisiran',
 'arterial hypertension, carotid artery, diabetes mellitus, hyperlipidemia, mediterranean diet, nicotine abstinence',
 'coronary calcium, statin',
 'coronary plaque regression, pcsk9 inhibitor, plaque stabilization',
 'cholesterol, colon cancer, diabetes, environmental aspect, heart attack, overweight, twitter',
 'niacin, pcsk9i',
 'inclisiran',
 'cv mortality, evolocumab, fourier trial open label extension',
 'inclisiran',
 'horizon trial, niacin, pelacarsen approval',
 'administration, inclisiran, pharm

In [190]:
word_list = list(filter(('').__ne__, data['keywords'].to_list()))
word_list

['true, picture, diabetes, heart, stroke, money, fund, feel, insufficient, funding',
 'endpoint, plaque, ivus, irritating, surrogate',
 'patient, inhibitor, reduction, benefit, ldl, chol, repatha, mortality, unprecedented, quit',
 'pruritis, mab, approved, patient, develop, marked, priority, wanted, continue, found',
 'fat, sugar, intake, fruit, evidence, bad, food, vegetable, linked, healthy, butter, diet, lifestyle, obesity, fatty, comment, response, saturated, trans, acid',
 'helpful, doc, approach, in-house, education',
 'lovastatin, doctor, tinyurl.com, natural, statin',
 'wiki, inclisiran, conference, happened, cardiovascular',
 'movement, tip, artery, nicotine, abstinence',
 'high, t.co, statin, ja.ma, medium',
 'pacman-ami, twitter.com, early, inhibitor, plaque, alirocumab, addition, high-intensity, statin, increase',
 'cholesterol, diabetes, cancer, overweight, attack, health, aspect, twitter, shown, eating',
 'niacin, isolated, art, medicine, pelacarsen, pasture, specific, of

In [182]:
stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [191]:
final_word_list=[]
for word in word_list:
    for w in word.split(", "):
        if w not in stopwords:
            final_word_list.append(w)

In [192]:
final_word_list

['true',
 'picture',
 'diabetes',
 'heart',
 'stroke',
 'money',
 'fund',
 'feel',
 'insufficient',
 'funding',
 'endpoint',
 'plaque',
 'ivus',
 'irritating',
 'surrogate',
 'patient',
 'inhibitor',
 'reduction',
 'benefit',
 'ldl',
 'chol',
 'repatha',
 'mortality',
 'unprecedented',
 'quit',
 'pruritis',
 'mab',
 'approved',
 'patient',
 'develop',
 'marked',
 'priority',
 'wanted',
 'continue',
 'fat',
 'sugar',
 'intake',
 'fruit',
 'evidence',
 'bad',
 'food',
 'vegetable',
 'linked',
 'healthy',
 'butter',
 'diet',
 'lifestyle',
 'obesity',
 'fatty',
 'comment',
 'response',
 'saturated',
 'trans',
 'acid',
 'helpful',
 'doc',
 'approach',
 'in-house',
 'education',
 'lovastatin',
 'doctor',
 'tinyurl.com',
 'natural',
 'statin',
 'wiki',
 'inclisiran',
 'conference',
 'happened',
 'cardiovascular',
 'movement',
 'artery',
 'nicotine',
 'abstinence',
 'high',
 't.co',
 'statin',
 'ja.ma',
 'medium',
 'pacman-ami',
 'twitter.com',
 'early',
 'inhibitor',
 'plaque',
 'alirocumab',

In [185]:
final_phrase_list=[]
for phrase in phrase_list:
    for w in phrase.split(", "):
        if w not in stopwords:
            final_phrase_list.append(w)

In [186]:
final_phrase_list

['diabetes',
 'guideline development',
 'heart attack',
 'public fund',
 'stroke',
 'plaque regression',
 'surrogate endpoint',
 'repatha',
 'antihistamine',
 'inclisiran',
 'pruritis',
 'cardiovascular disease',
 'chemical reaction',
 'dental damage',
 'diabetes',
 'dietitian',
 'healthy eating',
 'heart',
 'internet',
 'margarine',
 'monounsaturated fat',
 'obesity',
 'saturated fat',
 'sedentary behavior',
 'trans fatty acid',
 'type 2 diabetes',
 'cholesterol',
 'statin',
 'lovastatin',
 'red rice',
 'cardiovascular disease conference',
 'inclisiran',
 'arterial hypertension',
 'carotid artery',
 'diabetes mellitus',
 'hyperlipidemia',
 'mediterranean diet',
 'nicotine abstinence',
 'coronary calcium',
 'statin',
 'coronary plaque regression',
 'pcsk9 inhibitor',
 'plaque stabilization',
 'cholesterol',
 'colon cancer',
 'diabetes',
 'environmental aspect',
 'heart attack',
 'overweight',
 'twitter',
 'niacin',
 'pcsk9i',
 'inclisiran',
 'cv mortality',
 'evolocumab',
 'fourier tri

In [193]:
word_df=pd.DataFrame(final_word_list,columns=['Word'])
word_c=word_df['Word'].value_counts().to_dict()
word_dct = {k:[v] for k,v in word_c.items()}  # WORKAROUND
word_df2 = pd.DataFrame(word_dct).T
word_df2

0
inclisiran   172
t.co         131
twitter.com  115
evolocumab    76
statin        67
...          ...
scope          1
pod            1
bypass         1
in-house       1
eating         1

[1752 rows x 1 columns]

In [188]:
phrase_df=pd.DataFrame(final_phrase_list,columns=['Phrase'])
phrase_c=phrase_df['Phrase'].value_counts().to_dict()
phrase_dct = {k:[v] for k,v in phrase_c.items()}  # WORKAROUND
phrase_df2 = pd.DataFrame(phrase_dct).T
phrase_df2

0
inclisiran              185
evolocumab               63
alirocumab               46
statin                   29
bempedoic acid           25
...                     ...
lipidogists               1
atorv                     1
coronary heart disease    1
ct remnant                1
clinician                 1

[830 rows x 1 columns]

In [194]:
writer = pd.ExcelWriter('C:/Users/GOWRICH1/Downloads/Leqvio_Lexicon analysis01.19.23_HCP.xlsx', engine = 'xlsxwriter',options={'strings_to_urls': False})
data.to_excel(writer, index=False)
word_df2.to_excel(writer,sheet_name='All_Messages')
phrase_df2.to_excel(writer,sheet_name='Unique_Messages')
writer.save()
writer.close()